In [ ]:
import bn
import re
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os 
import numpy as np
from PIL import ImageFilter, ImageEnhance
from google import genai
from dotenv import load_dotenv

load_dotenv()

client = genai.Client()


In [10]:

#extract data pdf to text file
custom_config = r'--oem 3'

def enhance_image_for_ocr(image):
    gray = image.convert('L')
    contrast_enhancer = ImageEnhance.Contrast(gray)
    contrast = contrast_enhancer.enhance(2.0)
    sharp = contrast.filter(ImageFilter.SHARPEN)
    return sharp


def pdf_to_images(pdf_path, dpi=300):
    pages = convert_from_path(pdf_path, dpi=dpi)
    enhanced_pages = []
    for i in range(len(pages)):
        if i>=4 and i<=18:
            crop = pages[i].crop((0, 300, pages[i].width, pages[i].height-400))
            enhanced_page = enhance_image_for_ocr(crop)
            enhanced_pages.append(enhanced_page)
    return enhanced_pages

def extract_text_from_images(images):
    full_text = ""
    for i, img in enumerate(images):
        if i == 0:
            pass
        else:
            print(f"Processing page {i+1}...")
            text = pytesseract.image_to_string(img, lang='ben', config=custom_config)  # 'ben' = Bengali
            clean_text = text.replace('\xa0', ' ')  # Remove non-breaking spaces
            clean_text = "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])
            #clean_text = clean_text.replace('\n', ' ')
            #full_text += f"\n\n--- Page {i+1} ---\n\n{text}"
            full_text += clean_text
    return full_text


In [11]:
pdf = "HSC.pdf"  
question = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"

In [12]:
text_file = "extract_data2.txt"
if os.path.exists(text_file) == False:
    images = pdf_to_images(pdf_file)
    bangla_text = extract_text_from_images(images)
    with open(text_file, "w", encoding="utf-8") as f:
        f.write(bangla_text)

    print("✅ Done! Bangla text saved to 'extract_data2.txt'")
else:
    print("file exist please continue..........")

file exist please continue..........


In [13]:
def chunk_and_vectorize_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text_content = file.read()

        # --- Chunking ---
        # Tokenize the text into individual sentences
        text_content = text_content.replace("\n"," ")
        sentences = text_content.split("।")
        print(len(sentences))
        sum_sentence = []
        for i in range(0,len(sentences),8):
            chunk = sentences[i:i+8]
            sum_sentence.append("".join(chunk))
        return sum_sentence
    except:
        print("error")

def search_chunks_multiple_keywords(chunks, keywords):
    # Normalize keywords to lowercase
    keywords = [k.lower() for k in keywords]
    
    # Search for any of the keywords in each chunk
    result = []
    for chunk in chunks:
        chunk_lower = chunk.lower()
        if any(k in chunk_lower for k in keywords):
            result.append(chunk)
    return result


In [14]:
file_path = "extract_data2.txt"
chunks = chunk_and_vectorize_text(file_path)

371


In [15]:
def output_get(content,question):
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=f'{content} read this content and give {question} this question answer. only give answer',
    )
    return response.text

In [16]:
st=bn.remove_stopwords(question)
words = bn.tokenizer(st) # or bn.tokenizer(text, 'word')
found_chunks = search_chunks_multiple_keywords(chunks, words)
result_al = []
for i, chunk in enumerate(found_chunks):
    result_al.append(chunk)
result = output_get(result_al,question)
print(result)

কন্যাপিতা শম্ভুনাথবাবুকে সুপুরুষ বলা হয়েছে। তাঁর চেহারা ছিল এমন যে, ভিড়ের মধ্যে দেখলে সকলের আগে তাঁর উপরেই চোখ পড়ত।
